In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Preprocessing parameters
IMG_SIZE = 224
BATCH_SIZE = 32
batch_size=32
# Define data directories
train_data_dir = '/content/drive/MyDrive/major/train'
validation_data_dir = '/content/drive/MyDrive/major/valid'
test_data_dir = '/content/drive/MyDrive/major/test'

# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Load validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=None,  # No labels for test data
    shuffle=False  # Do not shuffle test data
)

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Combine base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


# Create a data generator for the test dataset
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Adjust according to your model's input size
    batch_size=batch_size,
    class_mode='binary',  # Change to 'categorical' if your model has multiple classes
    shuffle=False  # Important to keep the order of predictions consistent with labels
)

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_generator)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Found 2539 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


Epoch 1/20
79/79 [==============================] - 171s 2s/step - loss: 0.5901 - accuracy: 0.7104 - val_loss: 0.4937 - val_accuracy: 0.7396
Epoch 2/20
79/79 [==============================] - 169s 2s/step - loss: 0.5270 - accuracy: 0.7495 - val_loss: 0.4927 - val_accuracy: 0.7083
Epoch 3/20
79/79 [==============================] - 166s 2s/step - loss: 0.4900 - accuracy: 0.7643 - val_loss: 0.5435 - val_accuracy: 0.6771
Epoch 4/20
79/79 [==============================] - 166s 2s/step - loss: 0.4755 - accuracy: 0.7682 - val_loss: 0.4237 - val_accuracy: 0.8125
Epoch 5/20
79/79 [==============================] - 159s 2s/step - loss: 0.4710 - accuracy: 0.7762 - val_loss: 0.4351 - val_accuracy: 0.7396
Epoch 6/20
79/79 [==============================] - 160s 2s/step - loss: 0.4447 - accuracy: 0.7866 - val_loss: 0.4396 - val_accuracy: 0.7604
Epoch 7/20
79/79 [==============================] - 165s 2s/step - loss: 0.4467 - accuracy: 0.7942 - val_loss: 0.4611 - val_accuracy: 0.7812
Epoch 8/20
79